In [1]:
import os
import json
from openai import OpenAI
import pandas as pd
import random

In [2]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [3]:
articles = pd.read_csv("articles.csv")
sampled_articles = (
    articles.groupby("merged_category")
    .apply(lambda x: x.sample(n=500, random_state=0))
    .reset_index(drop=True)
)

C:\Users\ychaf\AppData\Local\Temp\ipykernel_8644\1328324694.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=500, random_state=0))


In [4]:
lines = []
system_inst = """
You are a helpful assistant that generates arabic legal questions based on legal articles provided by users.
Please ensure the questions are clear, concise, and relevant to the content of the articles.
Don't include any additional information or context beyond what is necessary for the question.
Ask the question directly.
"""

In [5]:
with open("moroccan_legal_qsts.jsonl", "w", encoding="utf-8") as f:
    for index, row in sampled_articles.iterrows():
        article = row["article_text"]
        nb_sentences = random.randint(1, 6)

        prompt = f"""تخيل أن لديك مشكلة قانونية أو تحتاج إلى نصيحة تتعلق بمحتوى المادة القانونية أدناه.
        بناءً على المادة القانونية، هل يمكنك إنشاء سؤال أو طرح حالة قانونية تحتوي على {nb_sentences} جمل قد يطرحها شخص ما للحصول على مشورة أو معلومات قانونية.
        المادة القانونية:
        {article}"""

        f.write(json.dumps({
            "custom_id": f"article_{index}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-5-nano",
                "messages": [
                    {"role": "system",
                     "content": system_inst},
                    {"role": "user", "content": prompt}
                ],
            }
        }) + "\n")

In [6]:
upload = client.files.create(
    file=open("moroccan_legal_qsts.jsonl", "rb"),
    purpose="batch"
)
print("Input file uploaded:", upload.id)

Input file uploaded: file-NwGLiYYsXwVVTiD8MJewPx


In [7]:
batch = client.batches.create(
    input_file_id=upload.id,
    endpoint="/v1/chat/completions",
    completion_window="24h"
)
print("Batch created with ID:", batch.id)

Batch created with ID: batch_696a7fd1a5c08190873f288e6824c4c4


In [16]:
status = client.batches.retrieve(batch.id)
print("Status:", status.status)

Status: completed


In [17]:
# if status.status == "completed":
#     output_file_id = status.output_file_id
#     content = client.files.content(output_file_id)
#     with open("openai_batch_output.jsonl", "wb") as out:
#         out.write(content)
#     print("Batch output written to openai_batch_output.jsonl")
# else:
#     print("Batch did not complete:", status)

In [37]:
# open jsonl file called moroccan_legal_qsts_generated.jsonl and read the contents
with open("moroccan_legal_qsts_generated.jsonl", "r", encoding="utf-8") as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        data = json.loads(line)
        generated_qst = data["response"]["body"]["choices"][0]["message"]["content"]
        sampled_articles.at[i, "generated_question"] = generated_qst

In [41]:
sampled_articles.head()

,category,file_name,article_number,article_text,ocr,merged_category,generated_question
0,المادة الأسرية,مدونة الأسرة,166المادة,تستمر الحضانة إلى بلوغ سن الرشد القانوني للذكر...,False,المادة الأسرية والشؤون الاجتماعية,هل يحق لمحضون عمره 16 عامًا، بعد انتهاء العلاق...
1,المادة الأسرية,مدونة الأسرة,1المادة,2المادة يطلق على هذا القانون اسم مدونة األ...,False,المادة الأسرية والشؤون الاجتماعية,رجل مغربي مسلم يتزوج امرأة مغربية يهودية وفقاً...
2,التقاعد,نظام رواتب التقاعد المدنية,1626الفصل,إليها. التي ينتمون الرتبةوالمستخدمون المرسمون ...,False,المادة الأسرية والشؤون الاجتماعية,موظف ينتمي إلى الرتبة والمستخدمين المرسمين وال...
3,المادة الاجتماعية,المجلس الأعلى لإنعاش التشغيل,3المادة,.بالشغل كلفةالم الحكومية السلطة إليها توجهه ال...,False,المادة الأسرية والشؤون الاجتماعية,توجه جهة حكومية إلى الأطراف المعنية بطلب تقديم...
4,المادة الأسرية,مدونة الأسرة,84المادة,توقيعه. تقديرها فترة الزواج والوضعية المالية ل...,False,المادة الأسرية والشؤون الاجتماعية,بعد الطلاق بسبب تعسّف الزوج، كيف تحسب المحكمة ...


In [40]:
sampled_articles.to_csv("generated_questions.csv", index=False)